In [1]:
import torch 
import torch.nn as nn 

import h5py, os
import numpy as np
from matplotlib import pyplot as plt
from functions import transforms as T
from torch.nn import functional as F
from functions.subsample import MaskFunc
from scipy.io import loadmat
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from math import exp

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'  # check whether a GPU is available
torch.cuda.empty_cache()

In [3]:
def show_slices(data, slice_nums, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)
        plt.axis('off')

In [4]:
class MRIDataset(DataLoader):
    def __init__(self, data_list, acceleration, center_fraction, use_seed):
        self.data_list = data_list
        self.acceleration = acceleration
        self.center_fraction = center_fraction
        self.use_seed = use_seed

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        subject_id = self.data_list[idx]
        return get_epoch_batch(subject_id, self.acceleration, self.center_fraction, self.use_seed)

In [5]:
def get_epoch_batch(subject_id, acc, center_fract, use_seed=True):
    ''' random select a few slices (batch_size) from each volume'''

    fname, rawdata_name, slice = subject_id  
    
    with h5py.File(rawdata_name, 'r') as data:
        rawdata = data['kspace'][slice]
                      
    slice_kspace = T.to_tensor(rawdata).unsqueeze(0)
    S, Ny, Nx, ps = slice_kspace.shape

    # apply random mask
    shape = np.array(slice_kspace.shape)
    # mask_func = MaskFunc(center_fractions=[center_fract], accelerations=[acc])
    mask_func = MaskFunc(center_fractions=center_fract, accelerations=acc)
    seed = None if not use_seed else tuple(map(ord, fname))
    mask = mask_func(shape, seed)
      
    # undersample
    masked_kspace = torch.where(mask == 0, torch.Tensor([0]), slice_kspace)
    masks = mask.repeat(S, Ny, 1, ps)

    img_gt, img_und = T.ifft2(slice_kspace), T.ifft2(masked_kspace)

    # perform data normalization which is important for network to learn useful features
    # during inference there is no ground truth image so use the zero-filled recon to normalize
    norm = T.complex_abs(img_und).max()
    if norm < 1e-6: norm = 1e-6
    
    # normalized data
    img_gt, img_und, rawdata_und = img_gt/norm, img_und/norm, masked_kspace/norm
        
    return img_gt.squeeze(0), img_und.squeeze(0), rawdata_und.squeeze(0), masks.squeeze(0), norm

In [6]:
def load_data_path(data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_and_val = ['train', 'val']
    limit = 60
    
        
    l = sorted(os.listdir(data_path))
    print(len(l))
    
    for i in range(len(train_and_val)):

        data_list[train_and_val[i]] = []
        
        if i == 0 : la = l[:limit]
        else : la = l[limit:]
        print(len(la))
    
        for fname in la:
                
            subject_data_path = os.path.join(data_path, fname)
                     
            if not os.path.isfile(subject_data_path): continue
            
            with h5py.File(subject_data_path, 'r') as data:
                num_slice = data['kspace'].shape[0]
                
            # the first 5 slices are mostly noise so it is better to exlude them
            if i == 1:
                data_list[train_and_val[i]] += [(fname, subject_data_path, slice) for slice in range(0, num_slice)]
            else:
                data_list[train_and_val[i]] += [(fname, subject_data_path, slice) for slice in range(17, 22)]
    
    return data_list  

In [7]:
#PREPARE THE DATA 
data_list = load_data_path('/data/local/NC2019MRI/train')
# slices, height, width = input_k.shape()

acc = [4,8]
cen_fract = [0.08, 0.04]
# acc = [4]
# cen_fract = [0.08]
seed = False # random masks for each slice 
num_workers = 10 # data loading is faster using a bigger number for num_workers. 0 means using one cpu to load data

def my_collate(batch):
    batch_len = len(batch)
    data = torch.ones(batch_len, 1, 320, 320)
    target_list = torch.ones(batch_len, 1, 320, 320)
    
    for batch_value in range(len(batch)):
        input = batch[batch_value][1]
        input = T.complex_abs(input)
        input = T.center_crop(input, (320, 320))
        data[batch_value, 0, :, :] = input
        
        target = batch[batch_value][0]
        target = T.complex_abs(target)
        target = T.center_crop(target, (320, 320))
        target_list[batch_value, 0, :, :] = target
    return (target_list, data, None, None, None)
    
# create data loader for training set. It applies same to validation set as well
train_dataset = MRIDataset(data_list['train'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=1, num_workers=num_workers, collate_fn=my_collate)




70
60
10


In [8]:
class ConvolutionalBlock(nn.Module):
    """
    2 lots of:
        3x3 convolution
        batch normalisation
        ReLu  
    """

    def __init__(self, in_chans, out_chans):
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans

        self.layers = nn.Sequential(
            
            nn.Conv2d(in_chans, out_chans, kernel_size=3, padding=1, padding_mode='zeros', bias=False),
            nn.BatchNorm2d(out_chans),
            nn.ReLU(inplace=True),
        )
        

    def forward(self, input):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]
        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        #print('input', input.shape)
        return self.layers(input)



In [9]:
class UNet(nn.Module):
    """
        Unet model
        
        Encoder:
            input -> Convolutional Block (output 1)
            
            maxpool(output 1) -> Convolutional Block (output 2)
            maxpool(output 2) -> Convolutional Block (output 3)
                        
            concat(output 2, deconv(output 3)) -> Convolutional Block (output 4)
            
            concat(output 1, deconv(output 4)) -> Convolutional Block (output 5)
            
            output5 -> 1x1 Convolution (output 6)
            
    """

    def __init__(self, input_chans, output_chans, chans, depth=3, level_len=1, drop_prob=0.5):
        """
        Args:
            in_chans (int): Number of channels in the input to the U-Net model.
            out_chans (int): Number of channels in the output to the U-Net model.
            chans (int): Number of output channels of the first convolution layer.
            num_pool_layers (int): Number of down-sampling and up-sampling layers.
            drop_prob (float): Dropout probability.
        """
        super().__init__()
        

        self.chans = chans
        self.depth = depth
        self.level_len = level_len
        self.encoding_layers = nn.ModuleList()
        self.dropout = nn.Dropout(drop_prob)
        
        out_chans = chans
        
        # First level
        self.encoding_layers.append(ConvolutionalBlock(input_chans, chans))
        
        for j in range(level_len):
            self.encoding_layers.append(ConvolutionalBlock(chans, chans))
            
        self.encoding_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        
        '''ENCODER'''
        for i in range(depth-2):
            in_chans = out_chans
            out_chans = int(out_chans * 2)
            self.encoding_layers.append(ConvolutionalBlock(in_chans, out_chans))
            
            for j in range(level_len):
                self.encoding_layers.append(ConvolutionalBlock(out_chans, out_chans))
                
            self.encoding_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            #self.encoding_layers.append(self.dropout)
            
                               
        in_chans = out_chans
        out_chans = int(out_chans * 2)
        self.encoding_layers.append(ConvolutionalBlock(in_chans, out_chans))
        
        for j in range(level_len):
            self.encoding_layers.append(ConvolutionalBlock(out_chans, out_chans))       
        
        
        
        '''DECODER'''
        self.decoding_layers = nn.ModuleList()
                    
                                    
        in_chans = out_chans
        out_chans = int(in_chans / 2)
                                    
        self.decoding_layers.append(nn.ConvTranspose2d(in_chans, out_chans, kernel_size=2, stride=2)) 
                                    
        for i in range(depth-2):
            
            self.decoding_layers.append(ConvolutionalBlock(in_chans, out_chans))
            
            for j in range(level_len):
                self.decoding_layers.append(ConvolutionalBlock(out_chans, out_chans))
                
            in_chans = out_chans
            out_chans = int(out_chans / 2)
            self.decoding_layers.append(nn.ConvTranspose2d(in_chans, out_chans, kernel_size=2, stride=2))                        
            #self.decoding_layers.append(self.dropout)
                                    
        self.decoding_layers.append(ConvolutionalBlock(in_chans, out_chans))
        
        for j in range(level_len): 
            self.decoding_layers.append(ConvolutionalBlock(out_chans, out_chans))
        
        '''FINAL CONV'''
        
        self.finalConv = nn.Conv2d(in_channels=out_chans, out_channels=output_chans, kernel_size=1)
        
       
        ''''''
        
        

    def forward(self, input):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]
        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        level_output = []
        index_to_save = 1
                                    
        '''ENCODE'''  
        
        for i, layer in enumerate(self.encoding_layers):
            input = layer(input)
            if i == index_to_save: 
                level_output.append(input)
                index_to_save += 2 + self.level_len             
                                    
                                    

        '''DECODE'''
        index_to_cat = 0
        for i, layer in enumerate(self.decoding_layers):
            input = layer(input)
            if i == index_to_cat:
                level_output.pop()
                index_to_cat += 2 + self.level_len  
                input =  torch.cat((input, level_output[-1]), dim=1)

        
        output = self.finalConv(input)

        return output 

In [10]:
import torch.optim as optim
torch.manual_seed(42)


In [11]:
#ssim loss
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()


def create_window(window_size, channel=1):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window


def ssim(img1, img2, window_size=11, window=None, size_average=True, full=False, val_range=None):
    # Value range can be different from 255. Other common ranges are 1 (sigmoid) and 2 (tanh).
    if val_range is None:
        if torch.max(img1) > 128:
            max_val = 255
        else:
            max_val = 1

        if torch.min(img1) < -0.5:
            min_val = -1
        else:
            min_val = 0
        L = max_val - min_val
    else:
        L = val_range

    padd = 0
    (_, channel, height, width) = img1.size()
    if window is None:
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channel).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=padd, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padd, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=padd, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padd, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padd, groups=channel) - mu1_mu2

    C1 = (0.01 * L) ** 2
    C2 = (0.03 * L) ** 2

    v1 = 2.0 * sigma12 + C2
    v2 = sigma1_sq + sigma2_sq + C2
    cs = torch.mean(v1 / v2)  # contrast sensitivity

    ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)

    if size_average:
        ret = ssim_map.mean()
    else:
        ret = ssim_map.mean(1).mean(1).mean(1)

    if full:
        return ret, cs
    return ret


# Classes to re-use window
class SSIM(torch.nn.Module):
    def __init__(self, window_size=11, size_average=True, val_range=None):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average

        # Assume 1 channel for SSIM
        self.channel = 1
        self.window = create_window(window_size)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()
        
        if channel == self.channel and self.window.type == img1.type:
            window = self.window
        else:
            window = create_window(self.window_size, channel).to(img1.device).type(img1.dtype)
            self.window = window
            self.channel = channel

        return 1 - ssim(img2, img1, window=window, window_size=self.window_size, size_average=self.size_average, val_range=img2.max())

In [12]:
# set learning rate
lr = 1e-4
wd = 0.0
#optimiser
#stochastic gradient descent (SGD)
# optimiser = optim.SGD(model.parameters(), lr=lr)
#optimiser = optim.Adam(model.parameters(), lr=lr)
# optimiser = torch.optim.RMSprop(model.parameters(), lr, weight_decay=wd)

In [13]:
ssim_loss = SSIM()

In [14]:
def save_model(path, loss):
    full_path = path + "-loss-" + str(loss) + '.h5'
    torch.save(model.state_dict(), full_path)

In [15]:
import csv 


In [16]:
def train_model(model, model_name):
    optimiser = optim.Adam(model.parameters(), lr=lr)
    file_name = 'loss_log_' + model_name + '.csv'
    file = open(file_name, 'a', newline='')
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Loss"])
    #train the network 

    # set number of epoches, i.e., number of times we iterate through the training set
    epoches = 150


    for epoch in range(epoches):
        model.train() 
        mean = []
        for iter, data in enumerate(train_loader):
            target, input, _, _, _ = data
            # print(input.shape)
            input = input.to(device)
            # print(input.shape)
            # print(target.shape)
            target = target.to(device)

            output = model(input)
            # print(output.shape)
            loss = ssim_loss(output, target)
            # loss = msssim_loss(output, target)
            # loss = F.l1_loss(output, target)
            mean.append(loss)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

        l = sum(mean)/len(mean)
        print("Epoch {}'s loss: {}".format(epoch, l))
        writer.writerow(["{}".format(epoch), "{}".format(l)])

        #if(epoch % 50 == 0):
            #save_model('models/model', str(l))

        # print("After training: \n", model.state_dict())



In [17]:
PATH = 'model_final.h5'
#torch.save(model.state_dict(), PATH)

In [18]:
torch.cuda.empty_cache()
#model.load_state_dict(torch.load('model_final.h5'))
#model.eval()

In [26]:
# create data loader for training set. It applies same to validation set as well
val_dataset = MRIDataset(data_list['val'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=1, num_workers=num_workers, collate_fn=my_collate)

In [30]:
from skimage.measure import compare_ssim 
def ssim_old(gt, pred):
    """ Compute Structural Similarity Index Metric (SSIM). """
    return compare_ssim(
        gt.transpose(1, 2, 0), pred.transpose(1, 2, 0), multichannel=True, data_range=gt.max()
    )

In [28]:

file = open('model_accuracy.csv', 'a', newline='')
writer = csv.writer(file)
writer.writerow(["Model", "Mean Score"])
    
def evaluate_model_accuracy(model, model_name):
    ssim_scores = []

    with torch.no_grad():
        for iteration, sample in enumerate(val_loader):

            img_gt, img_und, _, _, _  = sample
            D = img_gt

            C = img_und

            output = model(img_und.to(device)).cpu().numpy().squeeze()
            ssim_scores.append(ssim_old(D.squeeze(1).numpy(), output))

    numpy_ssims = np.array(ssim_scores)
    print("len of list", len(ssim_scores))
    print("Mean:", numpy_ssims.mean())
    writer.writerow([model_name, numpy_ssims.mean()])

In [22]:
#train and evaluate all the test models


In [38]:
# layer length 5, depth 5
model_l5_d5 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=5,
    level_len=5,
    drop_prob=0.5
).to(device)
train_model(model_l5_d5, 'l1_d5')
evaluate_model_accuracy(model_l5_d5, 'l1_d5')

Epoch 0's loss: 0.593559205532074
Epoch 1's loss: 0.45563122630119324
Epoch 2's loss: 0.4320314824581146
Epoch 3's loss: 0.42509132623672485
Epoch 4's loss: 0.4214414358139038
Epoch 5's loss: 0.41990789771080017
Epoch 6's loss: 0.42219078540802
Epoch 7's loss: 0.419140100479126
Epoch 8's loss: 0.410228431224823
Epoch 9's loss: 0.40630674362182617
Epoch 10's loss: 0.40993064641952515
Epoch 11's loss: 0.4066329598426819
Epoch 12's loss: 0.4068334102630615
Epoch 13's loss: 0.4061625003814697
Epoch 14's loss: 0.400149405002594
Epoch 15's loss: 0.40614572167396545
Epoch 16's loss: 0.39960360527038574
Epoch 17's loss: 0.40407001972198486
Epoch 18's loss: 0.3929222822189331
Epoch 19's loss: 0.4024587869644165
Epoch 20's loss: 0.3928792476654053
Epoch 21's loss: 0.3959886133670807
Epoch 22's loss: 0.39467230439186096
Epoch 23's loss: 0.3909370005130768
Epoch 24's loss: 0.39745622873306274
Epoch 25's loss: 0.3995010256767273
Epoch 26's loss: 0.3969588875770569
Epoch 27's loss: 0.392737030982971

/bham/modules/roots/neural-comp/2019-20/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """


len of list 27
Mean: 0.6117025230451661


In [23]:
# layer length 4, depth 5
"""TRAIN THIS ONCE FOR LONGER"""
model_l4_d5 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=5,
    level_len=4,
    drop_prob=0.5
).to(device)
train_model(model_l4_d5, 'l4_d5')
evaluate_model_accuracy(model_l4_d5, 'l4_d5')

PATH = 'model_l4_d5.h5'
torch.save(model_l4_d5.state_dict(), PATH)

Epoch 0's loss: 0.7200685143470764
Epoch 1's loss: 0.5121859312057495
Epoch 2's loss: 0.5020924806594849
Epoch 3's loss: 0.49254676699638367
Epoch 4's loss: 0.4998781085014343
Epoch 5's loss: 0.4925278425216675
Epoch 6's loss: 0.4898596405982971
Epoch 7's loss: 0.48185470700263977
Epoch 8's loss: 0.49131882190704346
Epoch 9's loss: 0.48503851890563965
Epoch 10's loss: 0.4745520055294037
Epoch 11's loss: 0.4774417281150818
Epoch 12's loss: 0.47726014256477356
Epoch 13's loss: 0.47114264965057373
Epoch 14's loss: 0.4735529124736786
Epoch 15's loss: 0.47476834058761597
Epoch 16's loss: 0.4654330611228943
Epoch 17's loss: 0.46604451537132263
Epoch 18's loss: 0.47081366181373596
Epoch 19's loss: 0.4720826745033264
Epoch 20's loss: 0.4588731527328491
Epoch 21's loss: 0.45933905243873596
Epoch 22's loss: 0.45915594696998596
Epoch 23's loss: 0.4575578272342682
Epoch 24's loss: 0.4578365087509155
Epoch 25's loss: 0.4575991928577423
Epoch 26's loss: 0.45410528779029846
Epoch 27's loss: 0.4513201

Epoch 223's loss: 0.4060031473636627
Epoch 224's loss: 0.4073120057582855
Epoch 225's loss: 0.407105028629303
Epoch 226's loss: 0.404532253742218
Epoch 227's loss: 0.4073323607444763
Epoch 228's loss: 0.410197913646698
Epoch 229's loss: 0.4095152020454407
Epoch 230's loss: 0.41085708141326904
Epoch 231's loss: 0.4037908911705017
Epoch 232's loss: 0.40368297696113586
Epoch 233's loss: 0.4062989056110382
Epoch 234's loss: 0.40581950545310974
Epoch 235's loss: 0.4049311578273773
Epoch 236's loss: 0.40484291315078735
Epoch 237's loss: 0.40132591128349304
Epoch 238's loss: 0.39890506863594055
Epoch 239's loss: 0.40362027287483215
Epoch 240's loss: 0.4015459716320038
Epoch 241's loss: 0.4000847339630127
Epoch 242's loss: 0.4023091793060303
Epoch 243's loss: 0.3965860605239868
Epoch 244's loss: 0.4061133861541748
Epoch 245's loss: 0.40106597542762756
Epoch 246's loss: 0.4014279842376709
Epoch 247's loss: 0.4077092409133911
Epoch 248's loss: 0.4057271182537079
Epoch 249's loss: 0.4027869701385

ValueError: axes don't match array

In [41]:
# layer length 3, depth 5
model_l3_d5 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=5,
    level_len=3,
    drop_prob=0.5
).to(device)
train_model(model_l3_d5, 'l3_d5')
evaluate_model_accuracy(model_l3_d5, 'l3_d5')

Epoch 0's loss: 0.6861812472343445
Epoch 1's loss: 0.4914252460002899
Epoch 2's loss: 0.44523292779922485
Epoch 3's loss: 0.4333905875682831
Epoch 4's loss: 0.4333963394165039
Epoch 5's loss: 0.4207882583141327
Epoch 6's loss: 0.42644599080085754
Epoch 7's loss: 0.4143199622631073
Epoch 8's loss: 0.4143598973751068
Epoch 9's loss: 0.4118671715259552
Epoch 10's loss: 0.40658843517303467
Epoch 11's loss: 0.41251176595687866
Epoch 12's loss: 0.40771958231925964
Epoch 13's loss: 0.4070192873477936
Epoch 14's loss: 0.413042277097702
Epoch 15's loss: 0.40546461939811707
Epoch 16's loss: 0.40594542026519775
Epoch 17's loss: 0.3982446491718292
Epoch 18's loss: 0.4043457508087158
Epoch 19's loss: 0.4107398986816406
Epoch 20's loss: 0.40556928515434265
Epoch 21's loss: 0.39911559224128723
Epoch 22's loss: 0.4036979377269745
Epoch 23's loss: 0.39606496691703796
Epoch 24's loss: 0.3926731050014496
Epoch 25's loss: 0.3900507390499115
Epoch 26's loss: 0.3974209725856781
Epoch 27's loss: 0.3995352089

/bham/modules/roots/neural-comp/2019-20/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """


len of list 27
Mean: 0.6124401789268296


In [42]:
# layer length 1, depth 5
model_l1_d5 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=5,
    level_len=1,
    drop_prob=0.5
).to(device)
train_model(model_l1_d5, 'l1_d5')
evaluate_model_accuracy(model_l1_d5, 'l1_d5')

Epoch 0's loss: 0.6958761811256409
Epoch 1's loss: 0.4832083582878113
Epoch 2's loss: 0.4453544020652771
Epoch 3's loss: 0.43137040734291077
Epoch 4's loss: 0.43589818477630615
Epoch 5's loss: 0.42162618041038513
Epoch 6's loss: 0.42356711626052856
Epoch 7's loss: 0.4183069169521332
Epoch 8's loss: 0.420474112033844
Epoch 9's loss: 0.41210806369781494
Epoch 10's loss: 0.4138564467430115
Epoch 11's loss: 0.414833664894104
Epoch 12's loss: 0.41527989506721497
Epoch 13's loss: 0.405396968126297
Epoch 14's loss: 0.4062202274799347
Epoch 15's loss: 0.40544354915618896
Epoch 16's loss: 0.39890050888061523
Epoch 17's loss: 0.4166163206100464
Epoch 18's loss: 0.41152653098106384
Epoch 19's loss: 0.40781769156455994
Epoch 20's loss: 0.4088135361671448
Epoch 21's loss: 0.403796911239624
Epoch 22's loss: 0.4010133147239685
Epoch 23's loss: 0.38907602429389954
Epoch 24's loss: 0.3986527919769287
Epoch 25's loss: 0.3960491120815277
Epoch 26's loss: 0.40393760800361633
Epoch 27's loss: 0.39868766069

/bham/modules/roots/neural-comp/2019-20/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """


len of list 27
Mean: 0.6216806885251814


In [31]:
torch.cuda.empty_cache()

In [30]:
# layer length 2, depth 9
model_l2_d9 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=9,
    level_len=2,
    drop_prob=0.5
).to(device)
train_model(model_l2_d9, 'l2_d9')
evaluate_model_accuracy(model_l2_d9, 'l2_d9')

RuntimeError: CUDA out of memory. Tried to allocate 144.00 MiB (GPU 0; 5.77 GiB total capacity; 4.82 GiB already allocated; 57.38 MiB free; 33.68 MiB cached)

In [23]:
# layer length 2, depth 7
model_l2_d7 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=7,
    level_len=2,
    drop_prob=0.5
).to(device)
train_model(model_l2_d7, 'l2_d7')
evaluate_model_accuracy(model_l2_d7, 'l2_d7')

Epoch 0's loss: 0.5725957155227661
Epoch 1's loss: 0.46198368072509766
Epoch 2's loss: 0.4504479765892029
Epoch 3's loss: 0.4377734959125519
Epoch 4's loss: 0.43150264024734497
Epoch 5's loss: 0.42619848251342773
Epoch 6's loss: 0.42615753412246704
Epoch 7's loss: 0.42151781916618347
Epoch 8's loss: 0.42288944125175476
Epoch 9's loss: 0.41863229870796204
Epoch 10's loss: 0.42209258675575256
Epoch 11's loss: 0.4109518229961395
Epoch 12's loss: 0.40941083431243896
Epoch 13's loss: 0.4063420295715332
Epoch 14's loss: 0.405103474855423
Epoch 15's loss: 0.40482044219970703
Epoch 16's loss: 0.4023895561695099
Epoch 17's loss: 0.4064387083053589
Epoch 18's loss: 0.40147143602371216
Epoch 19's loss: 0.40177610516548157
Epoch 20's loss: 0.39961495995521545
Epoch 21's loss: 0.3990129828453064
Epoch 22's loss: 0.39232245087623596
Epoch 23's loss: 0.3973141312599182
Epoch 24's loss: 0.38863417506217957
Epoch 25's loss: 0.39232122898101807
Epoch 26's loss: 0.39333975315093994
Epoch 27's loss: 0.396

/bham/modules/roots/neural-comp/2019-20/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """


len of list 37
Mean: 0.6283836738447159


In [23]:
# layer length 2, depth 5
model_l2_d5 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=5,
    level_len=2,
    drop_prob=0.5
).to(device)
train_model(model_l2_d5, 'l2_d5')
evaluate_model_accuracy(model_l2_d5, 'l2_d5')
PATH = 'model_l2_d5.h5'
torch.save(model_l2_d5.state_dict(), PATH)

Epoch 0's loss: 0.5632447004318237
Epoch 1's loss: 0.4994288682937622
Epoch 2's loss: 0.49361294507980347
Epoch 3's loss: 0.4819276034832001
Epoch 4's loss: 0.48144450783729553
Epoch 5's loss: 0.4730851948261261
Epoch 6's loss: 0.4611646234989166
Epoch 7's loss: 0.4565940499305725
Epoch 8's loss: 0.44895273447036743
Epoch 9's loss: 0.4485222399234772
Epoch 10's loss: 0.4509560763835907
Epoch 11's loss: 0.446892112493515
Epoch 12's loss: 0.4467083811759949
Epoch 13's loss: 0.4440436363220215
Epoch 14's loss: 0.4404855966567993
Epoch 15's loss: 0.4396642744541168
Epoch 16's loss: 0.4388129711151123
Epoch 17's loss: 0.434488445520401
Epoch 18's loss: 0.4317094087600708
Epoch 19's loss: 0.43622681498527527
Epoch 20's loss: 0.4321979880332947
Epoch 21's loss: 0.4364053010940552
Epoch 22's loss: 0.4246625006198883
Epoch 23's loss: 0.4267881512641907
Epoch 24's loss: 0.4279594123363495
Epoch 25's loss: 0.42643216252326965
Epoch 26's loss: 0.4319537580013275
Epoch 27's loss: 0.4291950166225433

ValueError: axes don't match array

In [31]:
evaluate_model_accuracy(model_l2_d5, 'l2_d5')



ValueError: axes don't match array

In [24]:
PATH = 'model_l2_d5.h5'
torch.save(model_l2_d5.state_dict(), PATH)

In [ ]:
model_l2_d3 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=3,
    level_len=2,
    drop_prob=0.5
).to(device)
train_model(model_l2_d3, 'l2_d3')
evaluate_model_accuracy(model_l2_d3, 'l2_d3')

In [ ]:
torch.cuda.empty_cache()
model_l10_d5 = UNet(
    input_chans=1,
    output_chans=1,
    chans=32,
    depth=5,
    level_len=10,
    drop_prob=0.5
).to(device)
train_model(model_l10_d5, 'l10_d5')
evaluate_model_accuracy(model_l10_d5, 'l10_d5')

Epoch 0's loss: 0.7463968992233276
Epoch 1's loss: 0.570790708065033
Epoch 2's loss: 0.5444494485855103
Epoch 3's loss: 0.5255780220031738
Epoch 4's loss: 0.5200080275535583
Epoch 5's loss: 0.504986047744751
Epoch 6's loss: 0.4987660348415375
Epoch 7's loss: 0.49269339442253113
Epoch 8's loss: 0.49166759848594666
Epoch 9's loss: 0.48942673206329346
Epoch 10's loss: 0.4754091501235962
Epoch 11's loss: 0.48027223348617554
Epoch 12's loss: 0.4729929268360138
Epoch 13's loss: 0.48082593083381653
Epoch 14's loss: 0.48229721188545227
Epoch 15's loss: 0.4832826852798462
Epoch 16's loss: 0.4777844250202179
Epoch 17's loss: 0.47195130586624146
Epoch 18's loss: 0.4713931381702423
Epoch 19's loss: 0.4702211022377014
Epoch 20's loss: 0.47077739238739014


In [ ]:
ssim_scores = []

with torch.no_grad():
    for iteration, sample in enumerate(val_loader):
    
        img_gt, img_und, _, _, _  = sample
        D = img_gt
        
        C = img_und
    
        output = model(img_und.to(device)).cpu().numpy().squeeze()
        ssim_scores.append(ssim_old(D.squeeze(1).numpy(), output))
            
numpy_ssims = np.array(ssim_scores)
print("len of list", len(ssim_scores))
print("Mean:", numpy_ssims.mean())

In [ ]:
with torch.no_grad():
    for iteration, sample in enumerate(val_loader):
    
        img_gt, img_und, _, _, _  = sample
        D = img_gt
        
        C = img_und
    
        output = model(img_und.to(device)).cpu().numpy().squeeze()
        show_slices([D.squeeze(1).numpy()[6], output[6]], [0, 1], cmap='gray')
        
        if iteration >= 3: break

In [ ]:
def save_reconstructions(reconstructions, out_dir, filename):
    """
    Saves the reconstructions from a model into h5 files that is appropriate for submission
    to the leaderboard.
    Args:
        reconstructions (dict[str, np.array]): A dictionary mapping input filenames to
            corresponding reconstructions (of shape num_slices x height x width).
        out_dir (pathlib.Path): Path to the output directory where the reconstructions
            should be saved.
    """
    # print(len(reconstructions))
    for fname, recons in reconstructions.items():
        subject_path = os.path.join(out_dir, filename)
        print(subject_path)
        with h5py.File(subject_path, 'a') as f:
            f.create_dataset(fname, data=recons)

In [ ]:
test_path = '/data/local/NC2019MRI/test/'
files = []
file_names = []

for r, d, f in os.walk(test_path):
    for file in f:
        files.append(os.path.join(r, file))
        file_names.append(file)

with torch.no_grad():
    for i in range(len(files)):
        with h5py.File(files[i],  "r") as hf:
            volume_kspace_4af = hf['kspace_4af'][()]
            volume_kspace_8af = hf['kspace_8af'][()]
            volume_kspace4 = T.to_tensor(volume_kspace_4af)
            volume_kspace8 = T.to_tensor(volume_kspace_8af) 
            _4af, _8af = T.ifft2(volume_kspace4), T.ifft2(volume_kspace8)
            norm_4af = T.complex_abs(_4af).max()
            if norm_4af < 1e-6: norm_4af = 1e-6
            norm_8af = T.complex_abs(_8af).max()
            if norm_8af < 1e-6: norm_8af = 1e-6
            _4af = _4af / norm_4af
            _8af = _8af / norm_8af
            _4af = T.complex_abs(_4af.squeeze(0))
            _4af = T.center_crop(_4af, (320, 320)).to(device)
            _8af = T.complex_abs(_8af.squeeze(0))
            _8af = T.center_crop(_8af, (320, 320)).to(device)

            recon_4af = model(_4af.unsqueeze(1)).squeeze(1).cpu()
            recon_8af = model(_8af.unsqueeze(1)).squeeze(1).cpu()
            # print(recon_4af.shape)
            reconstructions = {'recon_4af': recon_4af.numpy(), 'recon_8af': recon_8af.numpy()}
            out_dir = 'saved/' # where you want to save your result. 
            if not (os.path.exists(out_dir)): os.makedirs(out_dir)
            save_reconstructions(reconstructions, out_dir, file_names[i])
        

In [ ]:
affile_path = 'saved/file1000817.h5'

with h5py.File(file_path,  "r") as hf:
    img1 = hf['recon_4af']
    img2 = hf['recon_8af']
    print(img2.shape)
    
    show_slices([img1[20],img2[20]], [0, 1], cmap='gray')